In [1]:
import pandas as pd

In [2]:
task1_data = pd.read_csv("/data/yingfei/cancer_data/llm_prompt_data/llm_inference_data/test_zeroshot_simple_vicuna_labelled.csv")
print(task1_data.shape) # zero shot

(2312, 6)


In [3]:
task1_data.head()

,cell_id,prompt,answer,output_vic,prediction,overall
0,ACH-000006,Think step by step and decide in a single word...,Sensitive,1. ASXL1: This mutation is known to increase t...,Sensitive,Sensitive
1,ACH-000006,Think step by step and decide in a single word...,Resistant,1. ASXL1: This mutation is known to be associa...,NaN,Resistant
2,ACH-000006,Think step by step and decide in a single word...,Resistant,1. ASXL1: This mutation is known to be associa...,Mixed,Resistant
3,ACH-000006,Think step by step and decide in a single word...,Sensitive,1. ASXL1: This mutation is known to be associa...,Mixed,Sensitive
4,ACH-000006,Think step by step and decide in a single word...,Sensitive,1. ASXL1: This gene is involved in regulating ...,Resistant,Resistant


In [4]:
task1_data = task1_data.loc[task1_data.output_vic.notna()].copy()
task1_data.shape

(200, 6)

In [5]:
print(task1_data.output_vic[5]) # ??? -> manual label

1. ASXL1: ASXL1 is a histone methyltransferase that plays a role in regulating gene expression. It has been shown to interact with MTOR and inhibit its activity. Therefore, ASXL1 mutations may affect the sensitivity of the cell line to JW-7-52-1.
2. FLT3: FLT3 is a receptor tyrosine kinase that is involved in cell survival and proliferation. JW-7-52-1 has been shown to inhibit FLT3 activity, which may affect the sensitivity of the cell line to the drug.
3. MAP3K1: MAP3K1 is a kinase that activates the MAPK/ERK pathway. JW-7-75-1 has been shown to inhibit MAP3K1 activity, which may affect the sensitivity of the cell line to the drug.
4. NOTCH1: NOTCH1 is a receptor tyrosine kinase that is involved in cell differentiation and proliferation. JW-7-52-1 has been shown to inhibit NOTCH1 activity, which may affect the sensitivity of the cell line to the drug.
5. TP53: TP53 is a tumor suppressor gene that is involved in cell cycle regulation and DNA damage response. JW-7-52-1 has been shown to

In [6]:
task1_data.prediction.value_counts()

prediction
Sensitive    52
Resistant    50
Mixed         3
Name: count, dtype: int64

In [7]:
task1_data.overall.value_counts()

overall
Resistant    65
Sensitive    63
Mixed         5
Name: count, dtype: int64

In [10]:
task1_data.prediction.fillna("None", inplace = True)
task1_data.overall.fillna("None", inplace = True)

In [11]:
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score
accuracy_score(task1_data.answer, task1_data.prediction)

0.23

In [13]:
precision_score(task1_data.loc[task1_data.prediction.isin(["Sensitive", "Resistant"]), 'answer'],\
                task1_data.loc[task1_data.prediction.isin(["Sensitive", "Resistant"]), 'prediction'], pos_label = "Sensitive")

0.46153846153846156

In [14]:
recall_score(task1_data.loc[task1_data.prediction.isin(["Sensitive", "Resistant"]), 'answer'],\
                task1_data.loc[task1_data.prediction.isin(["Sensitive", "Resistant"]), 'prediction'], pos_label = "Sensitive")

0.46153846153846156

In [15]:
f1_score(task1_data.loc[task1_data.prediction.isin(["Sensitive", "Resistant"]), 'answer'],\
                task1_data.loc[task1_data.prediction.isin(["Sensitive", "Resistant"]), 'prediction'], pos_label = "Sensitive")

0.46153846153846156

In [16]:
confusion_matrix(task1_data.loc[task1_data.prediction.isin(["Sensitive", "Resistant"]), 'answer'],\
                task1_data.loc[task1_data.prediction.isin(["Sensitive", "Resistant"]), 'prediction'], labels =  ["Sensitive", "Resistant"])

array([[24, 28],
       [28, 22]])

In [17]:
accuracy_score(task1_data.answer, task1_data.overall)

0.32

In [19]:
precision_score(task1_data.loc[task1_data.overall.isin(["Sensitive", "Resistant"]), 'answer'],\
                task1_data.loc[task1_data.overall.isin(["Sensitive", "Resistant"]), 'overall'], pos_label = "Sensitive")

0.5079365079365079

In [20]:
recall_score(task1_data.loc[task1_data.overall.isin(["Sensitive", "Resistant"]), 'answer'],\
                task1_data.loc[task1_data.overall.isin(["Sensitive", "Resistant"]), 'overall'], pos_label = "Sensitive")

0.49230769230769234

In [21]:
f1_score(task1_data.loc[task1_data.overall.isin(["Sensitive", "Resistant"]), 'answer'],\
                task1_data.loc[task1_data.overall.isin(["Sensitive", "Resistant"]), 'overall'], pos_label = "Sensitive")

0.5

In [22]:
confusion_matrix(task1_data.loc[task1_data.overall.isin(["Sensitive", "Resistant"]), 'answer'],\
                task1_data.loc[task1_data.overall.isin(["Sensitive", "Resistant"]), 'overall'], labels =  ["Sensitive", "Resistant"])

array([[32, 33],
       [31, 32]])

In [24]:
fewshot_set = test_prompt_data.iloc[219:,:]

In [ ]:
def get_few_shot_drug_cell_sim(fewshot_set, num=5):
    fewshot = ''
    for i in range(num):#len(fewshot_set)):
        idx = np.random.randint(0,len(fewshot_set))
        answer = fewshot_set.iloc[idx].answer
        question = fewshot_set.iloc[idx].prompt.split("[Reasoning].")[1].replace('?',answer)#.replace("\n", "")
        fewshot += f'\n\nExample {i}:\n'
        fewshot += question#.strip()
    
    return fewshot.strip()